# **What is SVD?**

Singular Value Decomposition (SVD), a classical method from linear algebra is getting popular in the field of data science and machine learning. This popularity is because of its application in developing recommender systems. There are a lot of online user-centric applications such as video players, music players, e-commerce applications, etc., where users are recommended with further items to engage with. 

The Singular Value Decomposition (SVD), a method from linear algebra that has been generally used as a dimensionality reduction technique in machine learning. SVD is a matrix factorisation technique, which reduces the number of features of a dataset by reducing the space dimension from N-dimension to K-dimension (where K<N). In the context of the recommender system, the SVD is used as a collaborative filtering technique. It uses a matrix structure where each row represents a user, and each column represents an item. The elements of this matrix are the ratings that are given to items by users.

The factorisation of this matrix is done by the singular value decomposition. It finds factors of matrices from the factorisation of a high-level (user-item-rating) matrix. The singular value decomposition is a method of decomposing a matrix into three other matrices as given below:

image.png

 Where A is a m x n utility matrix, U is a m x r orthogonal left singular matrix, which represents the relationship between users and latent factors, S is a r x r diagonal matrix, which describes the strength of each latent factor and V is a r x n diagonal right singular matrix, which indicates the similarity between items and latent factors. The latent factors here are the characteristics of the items, for example, the genre of the music. The SVD decreases the dimension of the utility matrix A by extracting its latent factors. It maps each user and each item into a r-dimensional latent space. This mapping facilitates a clear representation of relationships between users and items.

## **Singular Value Decomposition (SVD) based Movie Recommendation**

Below is an implementation of singular value decomposition (SVD) based on collaborative filtering in the task of movie recommendation. This task is implemented in Python. For simplicity, the [MovieLens 1M Dataset](https://grouplens.org/datasets/movielens/1m/) has been used. This dataset has been chosen because it does not require any preprocessing as the main focus of this article is on SVD and recommender systems.

Import the required python libraries:

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn nltk gensim --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import numpy as np
import pandas as pd

Read the dataset from where it is downloaded in the system. It consists of two files ‘ratings.dat’ and ‘movies.dat’ which need to be read.

In [ ]:
data = pd.io.parsers.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/singular_value_decomposition/ratings.dat', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')

movie_data = pd.io.parsers.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::')

Create the rating matrix with rows as movies and columns as users.

In [ ]:
ratings_mat = np.ndarray(
    shape=(np.max(data.movie_id.values), np.max(data.user_id.values)),
    dtype=np.uint8)
ratings_mat[data.movie_id.values-1, data.user_id.values-1] = data.rating.values

Normalise the matrix.

In [ ]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

Compute the Singular Value Decomposition (SVD).

In [ ]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

Define a function to calculate the cosine similarity. Sort by most similar and return the top N results.

In [ ]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

Define a function to print top N similar movies.

In [ ]:
def print_similar_movies(movie_data, movie_id, top_indexes):
  print('Recommendations for {0}: \n'.format(
  movie_data[movie_data.movie_id == movie_id].title.values[0]))
  for id in top_indexes + 1:
      print(movie_data[movie_data.movie_id == id].title.values[0])

Initialise the value of k principal components, id of the movie as given in the dataset, and number of top elements to be printed.

In [ ]:
k = 50
movie_id = 10 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)

Print the top N similar movies.

In [ ]:
print_similar_movies(movie_data, movie_id, indexes)

# **Related Articles:**

> * [SVD in Recommender System](https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/)

> * [TF-IDF from Scratch in Python](https://analyticsindiamag.com/hands-on-implementation-of-tf-idf-from-scratch-in-python/)

> * [Continuous Bag of Words](https://analyticsindiamag.com/the-continuous-bag-of-words-cbow-model-in-nlp-hands-on-implementation-with-codes/)

> * [NLP Case Study of Documents Similarity](https://analyticsindiamag.com/nlp-case-study-identify/)

> * [Review Classification](https://analyticsindiamag.com/step-by-step-guide-to-reviews-classification-using-svc-naive-bayes-random-forest/)

> * [Multi Class Text Classification](https://analyticsindiamag.com/multi-class-text-classification-in-pytorch-using-torchtext/)
